In [33]:
# Import needed modules

import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from datetime import datetime
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import matplotlib.colors as colors
import geopandas as gpd
from urllib.request import urlretrieve
import requests
from glob import glob
import os
from datetime import datetime, timedelta
import json
import sys
import math
from shapely.geometry import Point
from shapely.geometry.polygon import LinearRing

In [2]:
# Establish directory locations 

parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
output_dir    = os.path.join(parent_dir, 'Output/sat_plot')
NEODASS_dir   = os.path.join(satellite_dir, 'NEODASS')

In [8]:
# Plotting preferences

# Global
min_lon = -35
max_lon = -5
min_lat = 55
max_lat = 66

# CHLA
CHLA_color           = 'YlGnBu_r'
CHLA_plot_as_log     = True # Default True
CHLA_plot_lim_max    = 10   # Default 10
CHLA_plot_lim_min    = 0.1  # Default 0.1

# SST
SST_color            = 'YlOrRd'
SST_plot_as_log      = False # Default False
SST_plot_lim_max     = 15    # Default 15
SST_plot_lim_min     = 5     # Default 5

# SSH
SSH_color            = 'RdBu'
SSH_plot_as_log      = False # Default False
SSH_plot_lim_max     = 0.5   # Default  0.5
SSH_plot_lim_min     = -0.5  # Default -0.5.



In [ ]:
# Function to calculate the length of one degree of longitude at a given latitude
def lon_deg_length(latitude):
    # Convert latitude from degrees to radians
    lat_rad = np.radians(latitude)
    
    # Earth radius in kilometers
    earth_radius_km = 6371.0
    
    # Calculate the length of one degree of longitude at the given latitude using the Haversine formula
    lon_deg_length_km = np.cos(lat_rad) * (2 * np.pi * earth_radius_km) / 360.0
    
    return lon_deg_length_km

# Function to create a circle geometry
def create_circle(lon, lat, radius_km):
    # Calculate the length of one degree of longitude at the given latitude
    lon_deg_length_km = lon_deg_length(lat)
    
    # Convert radius from km to degrees
    radius_deg = radius_km / lon_deg_length_km
    
    # Create a Point object for the center of the circle
    center_point = Point(lon, lat)
    
    # Create a LinearRing object representing the circle
    circle = center_point.buffer(radius_deg)
    
    return circle

In [4]:
# Retrieve float locations, load in bathymetry data

wmo_list = [4903532, 1902637]
#Float 1 = test float in the Icelandic Bassin
float_1_url = 'https://data-argo.ifremer.fr/dac/aoml/4903532/4903532_Sprof.nc'
#Float 2 = test float on Custard with glider next to it
float_2_url = 'https://data-argo.ifremer.fr/dac/coriolis/1902637/1902637_Sprof.nc'

#List the floats
floats_url = [float_1_url, float_2_url]

#Assign the local float directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
floats_dir = os.path.join(parent_dir, 'Data/Floats')

#Create floats filename
floats_filenames = []
for i in floats_url:
    filename = floats_dir + '/' + i.rsplit('/', 1)[1]
    floats_filenames.append(filename)

for url, filename in zip(floats_url, floats_filenames):
    urlretrieve(url, filename)

#We will loop through the float files, we create two dataframe. One with all the positions from both floats, and one with only the last positions from both floats.
#Since we know plot the last position of the float at the date of the plot, the second one is not needed.

#Create empty df
position_df = pd.DataFrame({'PROF_NUM' : str(), 'LONGITUDE' : [], 'LATITUDE' : [], 'float' : int()})
last_position_df = pd.DataFrame({'PROF_NUM' : str(), 'LONGITUDE' : [], 'LATITUDE' : [], 'float' : int()})

for file, wmo in zip(floats_filenames, wmo_list):
    dat = xr.open_dataset(file)
    dat = dat.rename({'CYCLE_NUMBER':'PROF_NUM'}).swap_dims({'N_PROF':'PROF_NUM'})
    temp_df = dat[['LONGITUDE', 'LATITUDE', 'JULD']].to_dataframe().reset_index()
    temp_df['float'] = wmo
    last_temp_df = temp_df[temp_df['JULD'] == max(temp_df['JULD'])]

    last_position_df = pd.concat([last_position_df, last_temp_df], ignore_index=True)
    position_df = pd.concat([position_df, temp_df], ignore_index=True)
    dat.close()
position_df = position_df[position_df['LATITUDE'] > min_lat]

######

# Load shape files for bathymetry:
#first_line_path = 'c:\\Users\\flapet\\OneDrive - NOC\\Documents\\NRT_viz\\biocarbon_nrt_data_viz/Data/ne_10m_bathymetry_all/ne_10m_bathymetry_J_1000.shp'
#second_line_path = 'c:\\Users\\flapet\\OneDrive - NOC\\Documents\\NRT_viz\\biocarbon_nrt_data_viz/Data/ne_10m_bathymetry_all/ne_10m_bathymetry_I_2000.shp'

first_line_path  = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\data\\bathymetry\\ne_10m_bathymetry_J_1000.shp'
second_line_path = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\data\\bathymetry\\ne_10m_bathymetry_I_2000.shp'

gdf_1000 = gpd.read_file(first_line_path)
gdf_2000 = gpd.read_file(second_line_path)

C:\Users\hanshil\AppData\Local\Temp\ipykernel_31716\1457570185.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  last_position_df = pd.concat([last_position_df, last_temp_df], ignore_index=True)
C:\Users\hanshil\AppData\Local\Temp\ipykernel_31716\1457570185.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  position_df = pd.concat([position_df, temp_df], ignore_index=True)


In [55]:
### Plotting most recent data from NEODASS

## List most recent files

# Get yesterday's date in the required format
yesterday = (datetime.now() - timedelta(days=2)).strftime('%Y%m%d')

# List to store matching file names
matching_files = []

# Loop through the files in the directory
for filename in os.listdir(NEODASS_dir):
    if filename.endswith('.nc') and f'-1d-{yesterday}_{yesterday}-' in filename:
        matching_files.append(filename)

print(matching_files)

## List variables from these files

# Dictionary to store files by their base names (excluding '_a_' or '_b_') and key variables
file_dict = {}

for file in matching_files:
    # Identify the base name by removing '_a' or '_b' if present
    base_name = file.replace('_a', '_').replace('_b', '_')
    if base_name not in file_dict:
        file_dict[base_name] = {}
    filepath  = os.path.join(NEODASS_dir, file)
    ds        = xr.open_dataset(filepath)
    variables = list(ds.data_vars.keys())
    
    # Find the key variable excluding 'longitude' and 'latitude'
    key_variable = None
    for var in variables:
        if 'longitude' not in var.lower() and 'latitude' not in var.lower():
            key_variable = var
            break
    
    if key_variable:
        file_dict[base_name][file] = key_variable
    
    ds.close()

# Print the results
for base_name, files in file_dict.items():
    for file, key_variable in files.items():
        print(f'File: {file}')
        print(f'Base Name: {base_name}')
        print(f'Key Variable: {key_variable}')


['cmems_sea_level-sla-1d-20240524_20240524-nrt.nc', 'olci_a_1km-CHL_OC4ME-1d-20240524_20240524-nrt.nc', 'olci_b_1km-CHL_OC4ME-1d-20240524_20240524-nrt.nc', 'slstr_a-sea_surface_temperature-1d-20240524_20240524-nrt.nc', 'slstr_b-sea_surface_temperature-1d-20240524_20240524-nrt.nc', 'viirs_noaa20-chl_oc5ci-1d-20240524_20240524-nrt.nc', 'viirs_suomi-chl_oc5_pic-1d-20240524_20240524-nrt.nc']
File: cmems_sea_level-sla-1d-20240524_20240524-nrt.nc
Base Name: cmems_sea_level-sla-1d-20240524_20240524-nrt.nc
Key Variable: sla
File: olci_a_1km-CHL_OC4ME-1d-20240524_20240524-nrt.nc
Base Name: olci__1km-CHL_OC4ME-1d-20240524_20240524-nrt.nc
Key Variable: CHL_OC4ME
File: olci_b_1km-CHL_OC4ME-1d-20240524_20240524-nrt.nc
Base Name: olci__1km-CHL_OC4ME-1d-20240524_20240524-nrt.nc
Key Variable: CHL_OC4ME
File: slstr_a-sea_surface_temperature-1d-20240524_20240524-nrt.nc
Base Name: slstr_-sea_surface_temperature-1d-20240524_20240524-nrt.nc
Key Variable: sea_surface_temperature
File: slstr_b-sea_surface_te

In [58]:
# Plotting loops

for base_name, files in file_dict.items():

    # Check if there are multiple files associated with the base name
    if len(files) > 1:

        # List of data variables to be combined
        data_vars = []
        
        # Loop through each file and collect data variables
        for file, data_var in files.items():
            # Perform data preparation for files with the same base name
            cur_data_var = xr.open_dataset(os.path.join(NEODASS_dir, f'{file}'))
            data_vars.append(cur_data_var[data_var])
        
        # Align data variables from all files
        aligned_data_vars = xr.align(*data_vars, join='outer')
        
        # Combine aligned data variables
        combined_data_var = aligned_data_vars[0]
        for var in aligned_data_vars[1:]:
            combined_data_var      = combined_data_var.combine_first(var)
            combined_data_var_mean = np.nanmean(combined_data_var, axis=0)
            date_of_plot = str(combined_data_var['time'].data[0])[:10]
            
    else: # Single satellite file

        for file, data_var in files.items():   
            cur_data_var = xr.open_dataset(os.path.join(NEODASS_dir, f'{file}'))
            date_of_plot = str(cur_data_var['time'].data[0])[:10]

    # Initialise figure
    fig = plt.figure(figsize=(20, 10))
    ax  = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
    ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())

    # Set plot settings based on variable
    var_name_lower = data_var.lower()
    if 'chl' in var_name_lower:
        units_nonstandard = cur_data_var[data_var].attrs.get('units_nonstandard')
        xx_plot_cbar_label = 'Chlorophyll (' + units_nonstandard + ')'
        xx_plot_min = CHLA_plot_lim_min
        xx_plot_max = CHLA_plot_lim_max
        color       = CHLA_color
        log_scaling = CHLA_plot_as_log
        xx_output_dir_name = 'Chla'
    elif 'temp' in var_name_lower:
        units_nonstandard = cur_data_var[data_var].attrs.get('units_nonstandard')
        if units_nonstandard == 'kelvin':
            degree_sign = u'\N{DEGREE SIGN}'
            xx_plot_cbar_label = 'Sea Surface Temperature ('+degree_sign+'C)'
            if len(files) > 1:
                combined_data_var_mean -= 273.15
            else:
                cur_data_var[data_var] = cur_data_var[data_var] - 273.15 # Convert from Kelvin to Celsius
            xx_plot_min = SST_plot_lim_min
            xx_plot_max = SST_plot_lim_max
            color       = SST_color
            log_scaling = SST_plot_as_log
            xx_output_dir_name = 'SST'
    elif 'sla' in var_name_lower:
        units_nonstandard = cur_data_var[data_var].attrs.get('units')
        xx_plot_cbar_label = 'Sea level anamoly (' + units_nonstandard + ')'
        xx_plot_min = SSH_plot_lim_min
        xx_plot_max = SSH_plot_lim_max
        color       = SSH_color
        log_scaling = SSH_plot_as_log
        xx_output_dir_name = 'SLA'
    if log_scaling:
        norm = colors.LogNorm(vmin = xx_plot_min, vmax = xx_plot_max)
    else:
        norm = colors.Normalize(vmin = xx_plot_min, vmax = xx_plot_max)

    if len(files) > 1:
    # Clip data for clean colormap
        NEODASS_data_to_plot_1day = np.clip(combined_data_var_mean.data,xx_plot_min,xx_plot_max)
    else:
        NEODASS_data_to_plot_1day = np.clip(cur_data_var[data_var].data[0,:],xx_plot_min,xx_plot_max)
        
    # Main colormesh plot
    im_1 = ax.pcolormesh(cur_data_var['longitude'].data, 
                         cur_data_var['latitude'].data, 
                         NEODASS_data_to_plot_1day, 
                         cmap = color, 
                         norm=norm,
                         transform=ccrs.PlateCarree())

    # Colorbar settings
    cbar = plt.colorbar(im_1, ax = ax, label=xx_plot_cbar_label)

    # Plot title
    plot_title  = f'{xx_output_dir_name}' + f' {date_of_plot}'
    ax.set_title(plot_title,fontsize = 24)

    # Filter the float position data to highlight the last position at the date of the plot
    last_pos_df = position_df[position_df['JULD'] <= date_of_plot]
    red_point   = last_pos_df[last_pos_df['JULD'] == max(last_pos_df['JULD'])]
    ax.add_feature(cfeature.COASTLINE)

    # Ploting the float positions
    sc  = ax.scatter(last_pos_df['LONGITUDE'], last_pos_df['LATITUDE'], transform=ccrs.PlateCarree(), c = 'black', zorder = 3) # Scatter plot of the previous positions
    sc2 = ax.scatter(red_point['LONGITUDE'], red_point['LATITUDE'], transform=ccrs.PlateCarree(), c = 'red', zorder = 4) # Scatter plot of the most recent position in red
    sc3 = ax.scatter(-24,60,transform=ccrs.PlateCarree(), edgecolors='black', facecolors='none', marker='s') # Mark deploy point

    circle_geometry = create_circle(-24, 60, 130)
    circle_patch = ax.add_geometries([circle_geometry], crs=ccrs.PlateCarree(), transform=ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    
    # Plot the bathymetry at 1000 and 2000m
    gdf_1000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.5, edgecolor='k', facecolor='none')
    gdf_2000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.3, edgecolor='k', facecolor='none')
    
    # Add Lat and Lon grid
    gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, crs = ccrs.PlateCarree())
    gl.top_labels   = False # suppress top labels
    gl.right_labels = False # suppress right labels
    
    # Create a proxy artist for the legend
    proxy = plt.Line2D([0], [0], linestyle='none', marker='s', color='black', markersize=10, markerfacecolor='none')
    # Add a legend with the proxy artist
    ax.legend([proxy], ['Deploy Point'], loc='lower left')

    # Save directory based on variable
    save_dir = os.path.join(output_dir, f'{xx_output_dir_name}{"_log" if log_scaling else ""}')

    # Check if the directory exists, if not create it
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Format filename
    filename = f'{xx_output_dir_name}{"_log" if log_scaling else ""}_{date_of_plot}_{base_name}.png'

    # Save the plot
    plt.savefig(os.path.join(save_dir, filename))
    
    plt.clf()
    plt.close()


C:\Users\hanshil\AppData\Local\Temp\ipykernel_31716\1324976813.py:24: RuntimeWarning: Mean of empty slice
  combined_data_var_mean = np.nanmean(combined_data_var, axis=0)
C:\Users\hanshil\AppData\Local\Temp\ipykernel_31716\1324976813.py:24: RuntimeWarning: Mean of empty slice
  combined_data_var_mean = np.nanmean(combined_data_var, axis=0)
